In [ ]:
# Import necessary libraries
import os
import torch
from diffusers import StableDiffusionPipeline
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Install required libraries
!pip install diffusers transformers accelerate --upgrade
!pip install huggingface_hub

In [ ]:
# Clone the repository and load the model from Hugging Face
from huggingface_hub import login
login('your_huggingface_token')  # Replace with your Hugging Face token

# Load the model
model_id = "Laxhar/noobai-XL-1.1"
pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Move the pipeline to GPU if available
if torch.cuda.is_available():
    pipeline.to('cuda')

In [ ]:
# Update tokenizer config to remove character limits
import json

def update_tokenizer_config(tokenizer_path):
    config_file = os.path.join(tokenizer_path, 'tokenizer_config.json')
    if os.path.exists(config_file):
        with open(config_file, 'r') as file:
            config = json.load(file)

        # Remove limits if present
        config.pop("max_length", None)
        config.pop("min_length", None)

        with open(config_file, 'w') as file:
            json.dump(config, file, indent=4)

update_tokenizer_config(os.path.join(model_id, 'tokenizer'))
update_tokenizer_config(os.path.join(model_id, 'tokenizer_2'))

In [ ]:
# Function to generate images
def generate_image(prompt, sampler_method="ddim", sampling_steps=50, seed=42, cfg_scale=7.5, hires_fix=False, scale_creativity_steps=5, width=512, height=512):
    """
    Function to generate images based on user inputs
    Args:
        prompt (str): Text prompt to generate image.
        sampler_method (str): Sampling method to use (e.g., ddim, pndm, k_euler, etc.).
        sampling_steps (int): Number of steps for sampling.
        seed (int): Random seed for reproducibility.
        cfg_scale (float): Scale for guidance.
        hires_fix (bool): Whether to apply high-resolution fix.
        scale_creativity_steps (int): Steps for creativity adjustment in high-res mode.
        width (int): Width of generated image.
        height (int): Height of generated image.

    Returns:
        Image: Generated image.
    """
    generator = torch.manual_seed(seed)

    pipeline.sampler_method = sampler_method  # Adjust sampler method

    # High-resolution fix
    if hires_fix:
        pipeline.scheduler.config.update({"scale_creativity_steps": scale_creativity_steps})

    # Generate image
    image = pipeline(
        prompt,
        num_inference_steps=sampling_steps,
        guidance_scale=cfg_scale,
        generator=generator,
        width=width,
        height=height
    ).images[0]

    return image

In [ ]:
# Example usage
from IPython.display import display
image = generate_image(
    prompt="A fantasy landscape with waterfalls and flying islands",
    sampler_method="k_euler",
    sampling_steps=100,
    seed=1234,
    cfg_scale=8.0,
    hires_fix=True,
    scale_creativity_steps=10,
    width=768,
    height=768
)
display(image)

In [ ]:
# Save image
def save_image(image, path="/content/drive/MyDrive/generated_image.png"):
    image.save(path)

save_image(image)